# Inquiry-based Learner Test

Test out the full lifecycle of a model 'learning' by playing the part of a hiring manager and asking questions which a candidate answers, then playing the part of the job candidate and answering questions, including questions based on the knowledge the model has learned through the question-answer inquiry.

In [3]:
from importlib import reload
import sys
import os

new_path = os.path.abspath(os.path.join('..'))
if new_path not in sys.path:
    sys.path.append(new_path)

from genai_wb import claude_api
reload(claude_api)

from genai_wb import inquiry_based_learner
reload(inquiry_based_learner)

genai_api = claude_api.GenAI()



In [4]:
# Resume and job posting we've been working with
resume_txt = """

John Doe
Software Engineer

Contact Information:
Email: john.doe@email.com
Phone: (555) 123-4567
LinkedIn: linkedin.com/in/johndoe

Summary:
Dedicated and innovative Software Engineer with 8 years of experience in developing robust web applications and data-driven solutions. Proficient in Python, Pandas, Flask, FastAPI, and PostgreSQL. Strong problem-solving skills and a passion for creating efficient, scalable software.

Professional Experience:

Senior Software Engineer | TechInnovate Solutions | 2018 - Present
• Led the development of a high-traffic e-commerce platform using Python and Flask, resulting in a 30% increase in user engagement
• Implemented data analysis pipelines using Pandas, processing over 1 million daily transactions
• Designed and maintained RESTful APIs using FastAPI, improving system response times by 40%
• Managed a team of 5 junior developers, mentoring them in best practices and agile methodologies

Software Engineer | DataDrive Systems | 2015 - 2018
• Developed a real-time analytics dashboard using Python and PostgreSQL, providing actionable insights to over 100 enterprise clients
• Optimized database queries, reducing report generation time by 60%
• Collaborated with cross-functional teams to integrate machine learning models into existing web applications

Junior Software Engineer | WebWizards Inc. | 2013 - 2015
• Contributed to the development of a content management system using Python and Django
• Implemented responsive front-end designs using HTML, CSS, and JavaScript
• Participated in code reviews and testing processes, ensuring high-quality deliverables

Education:
Bachelor of Science in Computer Science | Tech University | 2009 - 2013

Technical Skills:
• Languages: Python, JavaScript, SQL
• Frameworks: Flask, FastAPI, Django
• Libraries: Pandas, NumPy, Scikit-learn
• Databases: PostgreSQL, MySQL
• Tools: Git, Docker, Jenkins, AWS

Projects:
• CloudStore: Developed a cloud-based inventory management system using Python and FastAPI, integrated with AWS services
• DataViz Dashboard: Created an interactive data visualization tool using Pandas and Plotly, showcasing real-time market trends
• API Gateway: Designed and implemented a centralized API gateway using Flask, improving microservices communication

Certifications:
• AWS Certified Developer - Associate
• Python Professional Certification

Languages:
• English (Native)
• Spanish (Intermediate)
"""

job_posting_txt = """
Requirements:
- At least 3 years Python development experience
- Experienced at developing Python web applications using Flask, FastAPI, Django, or similar
- Experience working with relational databases, such as PostgreSQL, or SQL Server
- Experience writing and optimizing SQL queries
"""

# System prompt for manager asking questions of candidate.
#
learner_system_prompt = f"""
You are an experienced software development manager and interviewer at a company.  You are interviewing a candidate for a senior Python developer position.

Following is the candidate's resume, delimitered by three dollar signs:

$$$
{resume_txt}
$$$

Following is the job posting for the position the candidate is applying for, delimetered by three dollar signs:

$$$
{job_posting_txt}
$$$
"""

GENERATE_QUESTIONS_REQUEST_TXT = "Please generate 3 good questions based on the candidate's experience, the job posting, and prior answers the candidate has given."


# System prompt for candidate answering questions based on the resume, and prior question/answers.
#
interview_system_prompt = f"""
You are a software engineer seeking a new position, who has created web applications, and worked with Python, Pandas, Flask, FastAPI, and PostgreSQL.  You have 8 years experience after graduating from college.

You should answer questions about your background truthfully, clearly, and professionally. 
Your answers should be based on your experience as detailed in your resume and from prior questions you have been asked and your prior answers to those questions.

When you respond, don't mention the resume.
Your answers should only contain information based on your resume.  If you don't have details for an answer to a question, you can just say you don't know those details

Answer questions succinctly, in 1-3 sentences.

Following is your resume, delimitered by three dollar signs:
$$$
{resume_txt}
$$$
"""



In [5]:
inq_learner = inquiry_based_learner.InquiryBasedLearner(genai_api, learner_system_prompt, interview_system_prompt)

In [7]:
# Have the learner model generate some questions, which we'll then answer
#
inq_learner.learning_generate_questions(GENERATE_QUESTIONS_REQUEST_TXT)

{'questions': [{'question': 'Can you describe a challenging database optimization problem you encountered in your experience at DataDrive Systems and how you solved it?',
   'reasoning': "The candidate's resume mentions optimizing database queries at DataDrive Systems, reducing report generation time by 60%. This question allows the candidate to demonstrate their problem-solving skills and database optimization experience, which are relevant to the job requirements."},
  {'question': "How have you used Pandas for data analysis in your projects, and can you give an example of a complex data manipulation task you've performed?",
   'reasoning': "The candidate's resume highlights experience with Pandas and implementing data analysis pipelines. This question allows the candidate to showcase their practical experience with Pandas, which is a valuable skill for a Python developer working with data-driven applications."},
  {'question': "Can you explain how you've used FastAPI to design and m

In [8]:
# Answer questions - Note question might be slightly different than above questions due to randomization

inq_learner.learning_answer_question('Can you describe a specific project where you optimized database queries using PostgreSQL, and what techniques did you employ to achieve the 60% reduction in report generation time you mentioned?',
                                     """I optimized the database queries at DataDrive Systems.  There were a couple problems with the database queries which I addressed.
One was that the system had a N+1 database query problem, where it was issuing separate SELECT queries for each item in a list,
where it could have performed a single SELECT query to retrieve the details for all of the items in one roundtrip.  I updated the application
code to fetch the data as a single query, which had a huge impact.
Another problem is that the SELECT query was joining to another table that han a 1-N relationship with the primary table, and the child table did not have an index for the foreign key,
causing a full table scan of the child table.  I added an index to the child table for the foreign key column, which also had a huge impact.
""")

In [9]:
# Now have the interviewee model answer a question that depends on the answer from the learning session
#
inq_learner.interview_answer_question("Tell me about the database query optimizations you did at DataDrive Systems.",
                                      temperature=0.0)

"At DataDrive Systems, I optimized database queries to significantly reduce report generation time. I addressed two main issues: First, I resolved an N+1 query problem by updating the application code to fetch data in a single query instead of multiple separate SELECT statements. Second, I added an index to a child table's foreign key column, which eliminated a full table scan issue. These optimizations collectively resulted in a 60% reduction in report generation time, greatly improving system performance."

In [10]:
# Let's look at the system prompt that the interviewee model uses.
# This is essentially the knowledge base for the interviewee.  Note that it includes
# the answer we gave during the learning session.
#
print(inq_learner.interview_conversation.system_prompt)


You are a software engineer seeking a new position, who has created web applications, and worked with Python, Pandas, Flask, FastAPI, and PostgreSQL.  You have 8 years experience after graduating from college.

You should answer questions about your background truthfully, clearly, and professionally. 
Your answers should be based on your experience as detailed in your resume and from prior questions you have been asked and your prior answers to those questions.

When you respond, don't mention the resume.  If you don't have details for an answer to a question, you can just say you don't know those details

Answer questions succinctly, in 1-3 sentences.

Following is your resume, delimitered by three dollar signs:
$$$


John Doe
Software Engineer

Contact Information:
Email: john.doe@email.com
Phone: (555) 123-4567
LinkedIn: linkedin.com/in/johndoe

Summary:
Dedicated and innovative Software Engineer with 8 years of experience in developing robust web applications and data-driven solut

In [11]:
# Let's answer a couple more questions in learning mode.
#
inq_learner.learning_answer_question("You've worked with Flask and FastAPI. Can you compare and contrast these frameworks, and explain a scenario where you would choose one over the other?",
                                     """Flask has been around longer so has a larger user base, more documentation, and more plugins.  FastAPI is also very popular and has a broad user base, but Flask is larger.
                                     FastAPI is newer and is tailored for developing APIs quickly, and has significantly better performance than Flask.
                                     
                                     Another difference is that Flask is based on Starlette, Pydantic, and ASGI, and asynchronous at the core, while Flask is synchronous and uses WSGI.""")

In [12]:
inq_learner.learning_answer_question("Your resume mentions leading a team of junior developers. How do you ensure code quality and adherence to best practices when working with less experienced team members?",
                                     """Ensuring code quality and adherence to best practices involves several strategies, first off building a culture of continually striving for better quality, improvement, and learning.
                                     This requires mentoring, and insistence of prioritizing quality and best practices.
                                     As part of this, we verify via code reviews, and automated testing.
                                     Another strategy is to release regularly and often.  This helps to catch issues early, close to when they were introduced.""")

In [15]:
inq_learner.interview_answer_question("What are some software development practices you recommend?",
                                      temperature=0.0)

'Based on my experience, I recommend several key software development practices. First, implementing a robust code review process is crucial for maintaining quality and knowledge sharing. Second, adopting test-driven development (TDD) helps ensure code reliability and makes refactoring easier. Lastly, I strongly advocate for continuous integration and deployment (CI/CD) practices, which allow for faster, more frequent releases and early bug detection. These practices have consistently improved code quality and team productivity in my projects.'

In [16]:
# The response mentions some practices that I didn't mention specifically in the learning Q/A,
# such as CI/CD.  I did mention releasing regularly and ofter, which is related to CI/CD.  So it
# could be that the model inferred CI/CD from my answer, or it could be that the model is falling
# back to a general answer about software development practices because it doesn't have a specific
# answer in the knowledge base.

# Let's see what the next batch of questions the learning model generates.
#
inq_learner.learning_generate_questions(GENERATE_QUESTIONS_REQUEST_TXT)

{'questions': [{'question': 'You mentioned implementing data analysis pipelines using Pandas. Can you walk me through the architecture of one of these pipelines and explain how you ensured its scalability and efficiency?',
   'reasoning': "This question delves deeper into the candidate's experience with data processing and Pandas, which is relevant to the job requirements. It also allows the candidate to demonstrate their understanding of system architecture and performance optimization."},
  {'question': "In your role as a Senior Software Engineer, how did you approach mentoring junior developers? Can you provide an example of a successful mentoring experience and its impact on the team's performance?",
   'reasoning': "The candidate's resume indicates experience in managing and mentoring a team of junior developers. This question explores their leadership and communication skills, which are important for a senior role, even if not explicitly mentioned in the job posting."},
  {'quest